In [1]:
!nvidia-smi 

Wed Jan 22 12:47:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 68%   62C    P2   150W / 250W |   3996MiB / 11019MiB |     17%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:5E:00.0 Off |                  N/A |
| 31%   49C    P8    16W / 250W |     11MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
'''import sys
sys.stdout = open('/dev/stdout', 'w')'''

"import sys\nsys.stdout = open('/dev/stdout', 'w')"

In [3]:
!ls

01_CNN.ipynb				   ln_jan20_testresult_nl1_15-Copy2.csv
SC_CNN_v0.ipynb				   ln_jan20_testresult_nl1_15.csv
SC_CNN_v0.py				   ln_jan20_testresult_nl1_30.csv
SC_CNN_v0_deepSC_Ln_jan20.ipynb		   ln_jan21_testresult_nl1_b128s.csv
SC_CNN_v0_deepSC_Ln_jan21_nl1_test.ipynb   ln_jan21_testresult_nl1_b16s.csv
SC_CNN_v0_deepSC__newbn_jan22-Copy1.ipynb  ln_jan21_testresult_nl1_b32s.csv
SC_CNN_v0_deepSC__newbn_jan22.ipynb	   ln_jan21_testresult_nl1_b64s.csv
SC_CNN_v0_deepSC_dilation_jan22.ipynb	   mnist_4x
SC_CNN_v0_deepSC_jan22-b64s.ipynb	   mnist_4x.tar.gz
Untitled.ipynb				   mnist_png.tar
__pycache__				   mnist_sized
class_acc_no_size.csv			   mnist_sized.tar.gz
class_acc_sized.csv			   mnist_sized_trans.ipynb
data					   nohup.out
data.tar.gz				   parallel.py
deepsc_nln_jan21.pt			   sized_val
deepscln_jan20.pt			   sized_val.tar.gz
elastic_test.ipynb			   test.csv
ln_jan20_testresult.csv			   total_acc_no_size.csv
ln_jan20_testresult_9only.csv		   total_acc_sized.csv
ln_jan20_testresult

In [4]:
DATA_PATH = ''

In [5]:
import os

In [6]:
#!tar -zxvf 'mnist_4x.tar.gz'

In [7]:
INTERNAL_DATA_PATH = 'mnist_4x/'
INTERNAL_DATA_PATH_MAIN = 'mnist_4x/resized'
INTERNAL_DATA_PATH_OTHER = 'mnist_4x/centered'

In [8]:
import os
import fnmatch
 
# Get a list of all files in directory
for rootDir, subdirs, filenames in os.walk(INTERNAL_DATA_PATH):
    # Find the files that matches the given patterm
    for filename in fnmatch.filter(filenames, '.*'):
        try:
            #print(filename)
            os.remove(os.path.join(rootDir, filename))
        except OSError:
            print("Error while deleting file")

In [9]:
from time import sleep
from tqdm import tqdm
for _ in tqdm(range(10)):
    sleep(0.001)

100%|██████████| 10/10 [00:00<00:00, 897.75it/s]


In [10]:
#os.mkdir('data')

In [11]:
'''import cv2
import os

new_path = ['data/ds6']#['data/ds1','data/ds2','data/ds3','data/ds4','data/ds5']
img_size = [3]#[729,243,81,27,9]  

for l in range(1):
    os.mkdir(new_path[l])
    for i in os.listdir(INTERNAL_DATA_PATH_MAIN):
        os.mkdir(os.path.join(new_path[l],i))
        for j in os.listdir(os.path.join(INTERNAL_DATA_PATH_MAIN,i)):
            os.mkdir(os.path.join(new_path[l],i,j))
            for k in tqdm(os.listdir(os.path.join(INTERNAL_DATA_PATH_MAIN,i,j))):
                img_origin = cv2.imread(os.path.join(INTERNAL_DATA_PATH_MAIN,i,j,k))
                img_resized = cv2.resize(img_origin,(img_size[l],img_size[l]))
                cv2.imwrite(os.path.join(new_path[l],i,j,k),img_resized)'''

"import cv2\nimport os\n\nnew_path = ['data/ds6']#['data/ds1','data/ds2','data/ds3','data/ds4','data/ds5']\nimg_size = [3]#[729,243,81,27,9]  \n\nfor l in range(1):\n    os.mkdir(new_path[l])\n    for i in os.listdir(INTERNAL_DATA_PATH_MAIN):\n        os.mkdir(os.path.join(new_path[l],i))\n        for j in os.listdir(os.path.join(INTERNAL_DATA_PATH_MAIN,i)):\n            os.mkdir(os.path.join(new_path[l],i,j))\n            for k in tqdm(os.listdir(os.path.join(INTERNAL_DATA_PATH_MAIN,i,j))):\n                img_origin = cv2.imread(os.path.join(INTERNAL_DATA_PATH_MAIN,i,j,k))\n                img_resized = cv2.resize(img_origin,(img_size[l],img_size[l]))\n                cv2.imwrite(os.path.join(new_path[l],i,j,k),img_resized)"

In [12]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision
import torch
from torch.utils.data import Dataset, DataLoader
import os
import matplotlib.pyplot as plt
import numpy as np

########################################################################
batch_size = 64
########################################################################


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#net = Net()
#if torch.cuda.device_count() > 1:
#    net = nn.DataParallel(net)
#net.to(device)


In [15]:
class SizedMnistDataset(Dataset):
    def __init__(self,dir_list,transform=None):
        self.dir_list = dir_list
        self.transform = transform
        
        self.dataset_list = []
        for i in range(len(dir_list)):
            temp_dataset = datasets.ImageFolder(root=self.dir_list[i],transform = self.transform)
            self.dataset_list.append(temp_dataset)
        
    def __len__(self):
        return len(self.dataset_list[0])
    
    def __getitem__(self,idx):
        return self.dataset_list[0][idx],self.dataset_list[1][idx],self.dataset_list[2][idx],self.dataset_list[3][idx],self.dataset_list[4][idx]
    

In [16]:
new_path = ['data/ds2','data/ds3','data/ds4','data/ds5','data/ds6']

train_dataset_path = [x+"/training" for x in new_path]
test_dataset_path = [x+"/testing" for x in new_path]
test_dataset_path

['data/ds2/testing',
 'data/ds3/testing',
 'data/ds4/testing',
 'data/ds5/testing',
 'data/ds6/testing']

In [17]:
#img_size = [729,243,81,27,9]  

transform = transforms.Compose(
    [
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainMnistDataset = SizedMnistDataset(train_dataset_path,transform)
testMnistDataset = SizedMnistDataset(test_dataset_path,transform)

trainloader = torch.utils.data.DataLoader(trainMnistDataset,
                                          batch_size = batch_size, 
                                          shuffle=True,
                                         num_workers=8)

testloader = torch.utils.data.DataLoader(testMnistDataset,
                                          batch_size = batch_size, 
                                          shuffle=False,
                                        num_workers=8)


In [18]:
'''temptrainloader = torch.utils.data.DataLoader(temp_dataset,
                                          batch_size = 32, 
                                          shuffle=True)
for i,image in enumerate(temptrainloader):
    print(image[0].size())
    break'''

'temptrainloader = torch.utils.data.DataLoader(temp_dataset,\n                                          batch_size = 32, \n                                          shuffle=True)\nfor i,image in enumerate(temptrainloader):\n    print(image[0].size())\n    break'

In [19]:
'''def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

for j in range(10):
    for i,image in tqdm(enumerate(trainloader)):
        if(i==0):
            print(image[0][0].size())
            print(image[0][1])
            print(image[1][1])
            print(image[2][1])
            print(image[3][1])
            print(image[4][1])'''
    
'''imshow(image[0][0])
plt.show()
imshow(image[1][0])
plt.show()
imshow(image[2][0])
plt.show()
imshow(image[3][0])
plt.show()
imshow(image[4][0])
plt.show()'''


'imshow(image[0][0])\nplt.show()\nimshow(image[1][0])\nplt.show()\nimshow(image[2][0])\nplt.show()\nimshow(image[3][0])\nplt.show()\nimshow(image[4][0])\nplt.show()'

In [20]:
'''import matplotlib.pyplot as plt
import numpy as np

# 이미지를 보여주기 위한 함수


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# 학습용 이미지를 무작위로 가져오기
dataiter = iter(trainloader)
images = dataiter.next()

for i,images in enumerate(trainloader):
    print(images[0].shape)
    imshow(images[0][0][0])
    plt.show()
    imshow(images[0][0][1])
    plt.show()
    imshow(images[0][0][2])
    plt.show()
    imshow(images[0][0][3])
    plt.show()
    imshow(images[0][0][4])
    plt.show()
    break

# 이미지 보여주기
imshow(torchvision.utils.make_grid(images))
# 정답(label) 출력
print(' '.join('%5s' % labels[j] for j in range(4)))
plt.show()'''

"import matplotlib.pyplot as plt\nimport numpy as np\n\n# 이미지를 보여주기 위한 함수\n\n\ndef imshow(img):\n    img = img / 2 + 0.5     # unnormalize\n    npimg = img.numpy()\n    plt.imshow(np.transpose(npimg, (1, 2, 0)))\n\n\n# 학습용 이미지를 무작위로 가져오기\ndataiter = iter(trainloader)\nimages = dataiter.next()\n\nfor i,images in enumerate(trainloader):\n    print(images[0].shape)\n    imshow(images[0][0][0])\n    plt.show()\n    imshow(images[0][0][1])\n    plt.show()\n    imshow(images[0][0][2])\n    plt.show()\n    imshow(images[0][0][3])\n    plt.show()\n    imshow(images[0][0][4])\n    plt.show()\n    break\n\n# 이미지 보여주기\nimshow(torchvision.utils.make_grid(images))\n# 정답(label) 출력\nprint(' '.join('%5s' % labels[j] for j in range(4)))\nplt.show()"

In [21]:
import torch.nn as nn
import torch.nn.functional as F

class SCModule(nn.Module):
    def __init__(self,in_channels,out_channels,output_number):
        super(SCModule, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.output_number = output_number

        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride=9, padding=4,dilation=4)
        self.conv2 = nn.Conv2d(in_channels, out_channels, 3, stride=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels, out_channels, 1)
        
    def forward(self, x):
        #print(self.out_channels)
        if self.output_number == 3:
            x_1_1 = self.conv1(x[0])
            x_1_2 = self.conv1(x[1])
            x_1_3 = self.conv1(x[2])
            
            x_2_1 = self.conv2(x[1])
            x_2_2 = self.conv2(x[2])
            x_2_3 = self.conv2(x[3])
            
            x_3_1 = self.conv3(x[2])
            x_3_2 = self.conv3(x[3])
            x_3_3 = self.conv3(x[4])
            
            o_1_1 = torch.stack([x_1_1,x_2_1,x_3_1])
            o_1_1 = torch.sum(o_1_1,0)
            
            o_1_2 = torch.stack([F.conv2d(x[1], self.conv1.weight, bias=self.conv1.bias, stride=3, padding=4, dilation=4, groups=1),
                                 F.conv2d(x[2], self.conv2.weight, bias=self.conv2.bias, stride=1, padding=1, dilation=1, groups=1)])
            o_1_2 = torch.sum(o_1_2,0)
            
            o_1_3 = F.conv2d(x[2], self.conv1.weight, bias=self.conv1.bias, stride=1, padding=4, dilation=4, groups=1)
            
            o_2_1 = torch.stack([x_1_2,x_2_2,x_3_2])
            o_2_1 = torch.sum(o_2_1,0)
            
            o_2_2 = torch.stack([F.conv2d(x[2], self.conv1.weight, bias=self.conv1.bias, stride=3, padding=4, dilation=4, groups=1),
                                 F.conv2d(x[3], self.conv2.weight, bias=self.conv2.bias, stride=1, padding=1, dilation=1, groups=1)])
            o_2_2 = torch.sum(o_2_2,0)
            
            o_2_3 = F.conv2d(x[3], self.conv1.weight, bias=self.conv1.bias, stride=1, padding=4, dilation=4, groups=1)
            
            o_3_1 = torch.stack([x_1_3,x_2_3,x_3_3])
            o_3_1 = torch.sum(o_3_1,0)
            
            o_3_2 = torch.stack([F.conv2d(x[3], self.conv1.weight, bias=self.conv1.bias, stride=3, padding=4, dilation=4, groups=1),
                                 F.conv2d(x[4], self.conv2.weight, bias=self.conv2.bias, stride=1, padding=1, dilation=1, groups=1)])
            o_3_2 = torch.sum(o_3_2,0)
            
            o_3_3 = F.conv2d(x[4], self.conv1.weight, bias=self.conv1.bias, stride=1, padding=4, dilation=4, groups=1)
            
            return [torch.cat([o_1_1,o_1_2,o_1_3],dim=1),
                    torch.cat([o_2_1,o_2_2,o_2_3],dim=1),
                    torch.cat([o_3_1,o_3_2,o_3_3],dim=1)]
    
        if self.output_number == 1:
            outputs = []
            x_1_1 = self.conv1(x[0]) 
            x_1_2 = self.conv2(x[1]) 
            x_1_3 = self.conv3(x[2]) 
            #return [x_1_1,x_1_2,x_1_3]
            x_1 = torch.stack([x_1_1,x_1_2,x_1_3])
            x_1 = torch.sum(x_1,0)
            #print(x_1.size())
            #x_1 = torch.cat([x_1_1,x_1_2],dim=1)
            outputs.append(x_1)
            return outputs
        
            
        '''outputs = []
        for i in range(self.output_number):
            x_1_1 = self.conv1(x[0+i]) 
            x_1_2 = self.conv2(x[1+i]) 
            x_1_3 = self.conv3(x[2+i]) 
            #return [x_1_1,x_1_2,x_1_3]
            x_1 = torch.stack([x_1_1,x_1_2,x_1_3])
            x_1 = torch.sum(x_1,0)
            #print(x_1.size())
            #x_1 = torch.cat([x_1_1,x_1_2],dim=1)
            outputs.append(x_1)
        return outputs'''
    

    
#model = SCModule(3,6,3)

In [22]:
'''for i,(d1,d2,d3,d4,d5) in enumerate(zip(trainloader1,trainloader2,trainloader3,trainloader4,trainloader5)):
    print(d1[0][0].shape)
    imshow(d1[0][0])
    plt.show()
    imshow(d2[0][0])
    plt.show()
    imshow(d3[0][0])
    plt.show()
    imshow(d4[0][0])
    plt.show()
    imshow(d5[0][0])
    plt.show()

    print(d1[0].shape)

    o1,o2,o3 = model(d1[0],d2[0],d3[0],d4[0],d5[0])

    print(o1.shape)
    break
'''

'for i,(d1,d2,d3,d4,d5) in enumerate(zip(trainloader1,trainloader2,trainloader3,trainloader4,trainloader5)):\n    print(d1[0][0].shape)\n    imshow(d1[0][0])\n    plt.show()\n    imshow(d2[0][0])\n    plt.show()\n    imshow(d3[0][0])\n    plt.show()\n    imshow(d4[0][0])\n    plt.show()\n    imshow(d5[0][0])\n    plt.show()\n\n    print(d1[0].shape)\n\n    o1,o2,o3 = model(d1[0],d2[0],d3[0],d4[0],d5[0])\n\n    print(o1.shape)\n    break\n'

In [23]:
'''tensor = torch.tensor((), dtype=torch.float64,device='cuda')
pad2_1 = tensor.new_zeros((32,16,729,729))#.to('cuda')
pad2_1[:,:6,:,:].size()'''

"tensor = torch.tensor((), dtype=torch.float64,device='cuda')\npad2_1 = tensor.new_zeros((32,16,729,729))#.to('cuda')\npad2_1[:,:6,:,:].size()"

In [24]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self,batch_size):
        super(Net, self).__init__()
        self.sc1 = SCModule(3,6,3) 
        self.bn1_1 = nn.BatchNorm2d(18)
        self.bn1_2 = nn.BatchNorm2d(18)
        self.bn1_3 = nn.BatchNorm2d(18)
        self.sc2 = SCModule(18,10,3)
        self.bn2_1 = nn.BatchNorm2d(30)
        self.bn2_2 = nn.BatchNorm2d(30)
        self.bn2_3 = nn.BatchNorm2d(30)
        self.sc3 = SCModule(30,15,3)
        self.bn3_1 = nn.BatchNorm2d(45)
        self.bn3_2 = nn.BatchNorm2d(45)
        self.bn3_3 = nn.BatchNorm2d(45)
        self.sc4 = SCModule(45,20,3)
        self.bn4_1 = nn.BatchNorm2d(60)
        self.bn4_2 = nn.BatchNorm2d(60)
        self.bn4_3 = nn.BatchNorm2d(60)
        self.sc5 = SCModule(60,90,1)
        self.bn5 = nn.BatchNorm2d(90)
        #self.sc3 = SCModule(20,15,1)
        #self.fc1_1 = nn.Linear(6*81*81,500)
        #self.fc1_2 = nn.Linear(6*27*27,500)
        #self.fc1_3 = nn.Linear(6*9*9,500)
        
        self.fc2 = nn.Linear(90*3*3,120)
        #self.bnfc2 = nn.BatchNorm1d(120)
        #self.fc2 = nn.Linear(1500,120)
        self.fc3 = nn.Linear(120,84)
        #self.bnfc3 = nn.BatchNorm1d(84)
        self.fc4 = nn.Linear(84,10)
        
        tensor = torch.tensor((), dtype=torch.float)
 
        self.pad1 = tensor.new_zeros((batch_size,60,243,243))#.to('cuda')
        self.pad2 = tensor.new_zeros((batch_size,60,81,81))#.to('cuda')


    def forward(self, i1,i2,i3,i4,i5):
        self.pad1 = self.pad1.to(i1.device)
        self.pad2 = self.pad2.to(i1.device)
        x1_1,x1_2,x1_3 = self.sc1([i1,i2,i3,i4,i5])
        #x1_2,x1_3 = self.sc1([i2,i3,i4,i5])
        x1_1 = F.relu(self.bn1_1(x1_1))
        x1_2 = F.relu(self.bn1_2(x1_2))
        x1_3 = F.relu(self.bn1_3(x1_3))
        
        #print(x1_1)
        #print(self.pad1)
        bat_size = x1_1.size()[0]
        chan_size = x1_1.size()[1]
        x2_1,x2_2,x2_3 = self.sc2([self.pad1[:bat_size,:chan_size,:,:],self.pad2[:bat_size,:chan_size,:,:],x1_1,x1_2,x1_3])
        #x1_2,x1_3 = self.sc1([i2,i3,i4,i5])
        x2_1 = F.relu(self.bn2_1(x2_1))
        x2_2 = F.relu(self.bn2_2(x2_2))
        x2_3 = F.relu(self.bn2_3(x2_3))
        
        bat_size = x2_1.size()[0]
        chan_size = x2_1.size()[1]
        x3_1,x3_2,x3_3 = self.sc3([self.pad1[:bat_size,:chan_size,:,:],self.pad2[:bat_size,:chan_size,:,:],x2_1,x2_2,x2_3])
        #x1_2,x1_3 = self.sc1([i2,i3,i4,i5])
        x3_1 = F.relu(self.bn3_1(x3_1))
        x3_2 = F.relu(self.bn3_2(x3_2))
        x3_3 = F.relu(self.bn3_3(x3_3))
        
        bat_size = x3_1.size()[0]
        chan_size = x3_1.size()[1]
        x4_1,x4_2,x4_3 = self.sc4([self.pad1[:bat_size,:chan_size,:,:],self.pad2[:bat_size,:chan_size,:,:],x3_1,x3_2,x3_3])
        #x1_2,x1_3 = self.sc1([i2,i3,i4,i5])
        x4_1 = F.relu(self.bn4_1(x4_1))
        x4_2 = F.relu(self.bn4_2(x4_2))
        x4_3 = F.relu(self.bn4_3(x4_3))
    
        #print(x1_1.size())
        
        #x1_1 = x1_1.view(-1,6*81*81)
        #x1_2 = x1_2.view(-1,6*27*27)
        #x1_3 = x1_3.view(-1,6*9*9)
        
        #x1_1 = F.relu(self.fc1_1(x1_1))
        #x1_2 = F.relu(self.fc1_2(x1_2))
        #x1_3 = F.relu(self.fc1_3(x1_3))
        
        #x2 = torch.cat([x1_1,x1_2,x1_3],dim=1)
        
        x2 = self.sc5([x4_1,x4_2,x4_3])
        x2 = F.relu(self.bn5(x2[0]))
        
        #print(x2.size())
        
        #print(x2_1.size())
        
        #print(x3.size())
        
        x2 = x2.view(-1, 90 * 3 * 3)
        x2 = F.relu(self.fc2(x2))
        x2 = F.relu(self.fc3(x2))
        x2 = self.fc4(x2)
        #print(x4.shape)
        return x2

In [25]:
from parallel import DataParallelModel,DataParallelCriterion
from torch.nn.parallel.data_parallel import DataParallel
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_ids = [0,1]
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '29502'
dist.init_process_group(backend='nccl',rank=0,world_size=1)
model = Net(batch_size).to('cuda:1')
if torch.cuda.device_count() > 10:
    model = DDP(model,device_ids = [2,3])
#model.to(device)

In [26]:
print(device)

cuda


In [27]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#criterion = DataParallelCriterion(criterion,device_ids=[0,1])
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [28]:
SIZED_VAL_PATH = 'sized_val'
new_path = []
for i in os.listdir('mnist_sized'):
    #print(i)
    temp = []
    for j in range(1,6):
        temp.append(os.path.join(SIZED_VAL_PATH,str(i),'ds'+str(j)))
    new_path.append(temp)

new_path.sort(key=lambda x:int(x[0].split('/')[1]))

val_gen_arr = []
for i in tqdm(range(0,len(new_path))):
    transform = transforms.Compose(
        [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    testMnistDataset = SizedMnistDataset(new_path[i],transform)

    testloader = torch.utils.data.DataLoader(testMnistDataset,
                                              batch_size = batch_size, 
                                              shuffle=False,
                                            num_workers=8)
    val_gen_arr.append(testloader)
#
#val_gen_arr
accuracy = []


100%|██████████| 112/112 [00:01<00:00, 67.86it/s]


In [29]:
import torch.optim as optim
from tqdm import tqdm
import sys

trn_loss_list = []
val_loss_list = []
acc_list = []
num_batches = len(trainloader)
for epoch in range(30):
    running_loss = 0.0
    model.train()
    device = 'cuda:1'
    for i,images in tqdm(enumerate(trainloader)):
        #data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        c1 = images[0][0].to(device)
        c2 = images[1][0].to(device)
        c3 = images[2][0].to(device)
        c4 = images[3][0].to(device)
        c5 = images[4][0].to(device)
        c6 = images[0][1].to(device)
        #c61 = c6[:128].to('cuda:0')
        #c62 = c6[128:].to('cuda:1')
        output = model(c1,c2,c3,c4,c5)
        #output = torch.nn.parallel.gather(output,'cuda:1')
        loss = criterion(output, c6)
        loss.backward()    # calc gradients
        optimizer.step()   # update gradients
        running_loss += loss.item()
        #print(i)
    
    model.eval()
    with torch.no_grad(): # very very very very important!!!
        val_loss = 0.0
        class_correct = list(0. for i in range(10))
        class_total = list(0. for i in range(10))
        correct = 0
        total = 0
        for j,val in enumerate(testloader):
            v1 = val[0][0].to(device)
            v2 = val[1][0].to(device)
            v3 = val[2][0].to(device)
            v4 = val[3][0].to(device)
            v5 = val[4][0].to(device)
            val_labels = val[0][1].to(device)
            val_output = model(v1,v2,v3,v4,v5)
            v_loss = criterion(val_output, val_labels)
            val_loss += v_loss
            _, predicted = torch.max(val_output, 1)
            c = (predicted == val_labels).squeeze()
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()
            for i in range(len(val_labels)):
                val_label = val_labels[i]
                class_correct[val_label] += c[i].item()
                class_total[val_label] += 1

        for i in range(10):
            if class_total[i]==0:
                print('class_total = 0',class_correct,class_total)
            else:
                print('Accuracy of %5s : %2d %%' % (i, 100 * class_correct[i] / class_total[i]))

    print("epoch: {}/{} | step: {}/{} | trn loss: {:.4f} | val loss: {:.4f}".format(
        epoch+1, 30, i+1, num_batches, running_loss / len(trainloader), val_loss / len(testloader)
    ))        
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

    trn_loss_list.append(running_loss/1875)
    val_loss_list.append(val_loss/len(testloader))
    running_loss = 0.0
    
    temp_acc = []
    for testloader in tqdm(val_gen_arr):
        correct = 0
        total = 0
        class_correct = list(0. for i in range(10))
        class_total = list(0. for i in range(10))
        with torch.no_grad():
            for images in testloader:
                c1 = images[0][0].to(device)
                c2 = images[1][0].to(device)
                c3 = images[2][0].to(device)
                c4 = images[3][0].to(device)
                c5 = images[4][0].to(device)
                val_labels = images[0][1].to(device)
                outputs = model(c1,c2,c3,c4,c5)
                _, predicted = torch.max(outputs.data, 1)
                c = (predicted == val_labels).squeeze()
                total += val_labels.size(0)
                correct += (predicted == val_labels).sum().item()
        #print(total,correct,end='')

        print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
        temp_acc.append(100 * correct / total)
    acc_list.append(temp_acc)
        
    #optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    

938it [01:13, 12.76it/s]
  0%|          | 0/112 [00:00<?, ?it/s]

Accuracy of     0 : 98 %
Accuracy of     1 : 100 %
Accuracy of     2 : 99 %
Accuracy of     3 : 99 %
Accuracy of     4 : 100 %
Accuracy of     5 : 97 %
Accuracy of     6 : 96 %
Accuracy of     7 : 99 %
Accuracy of     8 : 98 %
Accuracy of     9 : 97 %
epoch: 1/30 | step: 10/938 | trn loss: 0.1379 | val loss: 0.0566
Accuracy of the network on the 10000 test images: 98 %


  1%|          | 1/112 [00:01<02:41,  1.45s/it]

Accuracy of the network on the 10000 test images: 10 %


  2%|▏         | 2/112 [00:02<02:38,  1.44s/it]

...

Accuracy of the network on the 10000 test images: 99 %


 99%|█████████▉| 111/112 [02:43<00:01,  1.52s/it]

Accuracy of the network on the 10000 test images: 99 %


100%|██████████| 112/112 [02:45<00:00,  1.48s/it]

Accuracy of the network on the 10000 test images: 99 %


In [30]:
import csv

csvfile = open('testresult_dil_jan22.csv','w',newline="")

csvwriter = csv.writer(csvfile)
for row in acc_list:
    csvwriter.writerow(row)
    
csvfile.close()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(acc_list)
plt.show()

In [ ]:
acc_list

In [ ]:
model.sc1.conv1.weight

In [ ]:
model.eval()

In [ ]:
len(trainloader)

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for images in tqdm(testloader):
        c1 = images[0][0].to(device)
        c2 = images[1][0].to(device)
        c3 = images[2][0].to(device)
        c4 = images[3][0].to(device)
        c5 = images[4][0].to(device)
        labels = images[0][1].to(device)
        outputs = model(c1,c2,c3,c4,c5)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(total,correct,end='')

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

In [ ]:
c1.size()[0]

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for images in tqdm(testloader):
        c1 = images[0][0].to(device)
        c2 = images[1][0].to(device)
        c3 = images[2][0].to(device)
        c4 = images[3][0].to(device)
        c5 = images[4][0].to(device)
        labels = images[0][1].to(device)
        outputs = model(c1,c2,c3,c4,c5)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (i, 100 * class_correct[i] / class_total[i]))

In [ ]:
for i in range(10):
    print('Accuracy of %5s : %2d %%' % (i, 100 * class_correct[i] / class_total[i]))

In [ ]:
labels

In [ ]:
output.size()

In [ ]:
c6.size()

In [ ]:
SIZED_VAL_PATH = 'sized_val'

new_path = []

for i in os.listdir('mnist_sized'):
    #print(i)
    temp = []
    for j in range(1,6):
        temp.append(os.path.join(SIZED_VAL_PATH,str(i),'ds'+str(j)))
    new_path.append(temp)

#new_path

In [ ]:
new_path.sort(key=lambda x:int(x[0].split('/')[1]))
new_path

In [ ]:
val_gen_arr = []
for i in tqdm(range(len(new_path))):
    transform = transforms.Compose(
        [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    testMnistDataset = SizedMnistDataset(new_path[i],transform)

    testloader = torch.utils.data.DataLoader(testMnistDataset,
                                              batch_size = batch_size, 
                                              shuffle=False,
                                            num_workers=8,
                                            pin_memory=True)
    val_gen_arr.append(testloader)

In [ ]:
#val_gen_arr

In [ ]:
accuracy = []

for testloader in tqdm(val_gen_arr):
    correct = 0
    total = 0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for images in testloader:
            c1 = images[0][0].to(device)
            c2 = images[1][0].to(device)
            c3 = images[2][0].to(device)
            c4 = images[3][0].to(device)
            c5 = images[4][0].to(device)
            val_labels = images[0][1].to(device)
            outputs = model(c1,c2,c3,c4,c5)
            _, predicted = torch.max(outputs.data, 1)
            c = (predicted == val_labels).squeeze()
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()
            for i in range(len(val_labels)):
                val_label = val_labels[i]
                class_correct[val_label] += c[i].item()
                class_total[val_label] += 1

        for i in range(10):
            if class_total[i]==0:
                print('class_total = 0',class_correct,class_total)
            else:
                print('Accuracy of %5s : %2d %%' % (i, 100 * class_correct[i] / class_total[i]))
            #print(total,correct,end='')

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    accuracy.append((100 * correct / total))

In [ ]:
for i in range(len(accuracy)):
    print(i,accuracy[i])

In [ ]:
for i in range(len(accuracy)):
    print(accuracy[i],',',end='')

In [ ]:
torch.save(model.state_dict(),'deepscln_jan20.pt')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(accuracy)
plt.show()

In [ ]:
import csv

csvfile = open('test.csv','w',newline="")

csvwriter = csv.writer(csvfile)
for row in [accuracy]:
    csvwriter.writerow(row)
    
csvfile.close()